# Learning Looker SDK with Python

This notebook demonstrates how to use the `looker-sdk` directly to interact with dashboards, elements, and queries.

## 1. Setup
Import the SDK and initialize it. Ensure you have a `.env` file with your Looker credentials (`LOOKERSDK_BASE_URL`, `LOOKERSDK_CLIENT_ID`, `LOOKERSDK_CLIENT_SECRET`).

In [ ]:
import os
import looker_sdk
from dotenv import load_dotenv
import pandas as pd
import json

# Load environment variables
load_dotenv()

# Initialize Looker SDK (using 4.0 API)
sdk = looker_sdk.init40()

## 2. Get Dashboard and Elements

Use `sdk.dashboard(dashboard_id)` to get the dashboard object. The elements are located in the `dashboard_elements` property.

In [ ]:
DASHBOARD_ID = "YOUR_DASHBOARD_ID"  # Replace with your dashboard ID

# Fetch the dashboard
dashboard = sdk.dashboard(dashboard_id=DASHBOARD_ID)
elements = dashboard.dashboard_elements

print(f"Dashboard Title: {dashboard.title}")
print(f"Number of elements: {len(elements)}")

# List element titles and check for queries
for i, element in enumerate(elements):
    has_query = "Yes" if element.query_id else "No"
    print(f"[{i}] Title: {element.title} | Query ID: {element.query_id} | Has Query Object: {has_query}")

## 3. Create a Query from an Element

If you want to create a *new* query using the definition from a dashboard element's query property, you can use `sdk.create_query`. 

Note: We use `looker_sdk.models40.WriteQuery` to map the existing query properties to a new query request.

In [ ]:
# Pick an element that has a query
target_element = elements[0] # Adjust index as needed
q = target_element.query

if q:
    # Extract query definitions into a WriteQuery model
    new_query_body = looker_sdk.models40.WriteQuery(
        model=q.model,
        view=q.view,
        fields=q.fields,
        filters=q.filters,
        sorts=q.sorts,
        limit=q.limit,
        vis_config=q.vis_config,
        # You can add more fields if needed
    )
    
    # Use the sdk directly to create the query
    created_query = sdk.create_query(body=new_query_body)
    print(f"Created New Query ID: {created_query.id}")
else:
    print("Target element has no query object to copy.")

## 4. Run a Query

Use `sdk.run_query(query_id, result_format)` to get the data. You can run the query attached to the element directly, or run the new one we just created.

In [ ]:
QUERY_ID = target_element.query_id # Or use created_query.id

# Run the query and get results as JSON string
json_results = sdk.run_query(query_id=str(QUERY_ID), result_format="json")

# Load into Pandas for analysis
df = pd.read_json(json_results)
df.head()